In [1]:
from sklearn import *
import lightgbm as lgbm
import pandas as pd
import numpy as np
import re
import nltk
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
nltk.download("stopwords")
plt.style.use('ggplot')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/charubaiel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

# pos \ neg - комменты с твиттера http://study.mokoron.com/ 
# labeled каггловский датасет по токсикам https://www.kaggle.com/blackmoon/russian-language-toxic-comments

In [27]:
twitter_neg = pd.read_csv('data/negative.csv',sep=';',header=None,usecols=[3])
twitter_pos = pd.read_csv('data/positive.csv',sep=';',header=None,usecols=[3])
vk_all = pd.read_csv('data/labeled.csv')

In [47]:
ttl_toxic = vk_all.append(twitter_pos.rename(columns={3:'comment'}).sample(5000)).fillna(0)


In [29]:
ttl_toxic['toxic'].value_counts(normalize=True)

0.0    0.751391
1.0    0.248609
Name: toxic, dtype: float64

In [30]:
scores = {}

In [31]:
data,val_data,target,val_target = model_selection.train_test_split(ttl_toxic['comment'],ttl_toxic['toxic'],train_size=.75,stratify=ttl_toxic['toxic'])

### stupid baseline

In [8]:
tf = feature_extraction.text.TfidfVectorizer()

In [9]:
tf.fit(data)

TfidfVectorizer()

In [10]:
clf_base = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight=target.value_counts().to_dict())
pipe_base = pipeline.make_pipeline(tf,clf_base)

In [11]:
scores['baseline'] = pd.DataFrame(model_selection.cross_validate(pipe_base,data,target,scoring=['f1','precision','recall'],cv=5,verbose=10)).mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] START .....................................................................
[CV] END  f1: (test=0.692) precision: (test=0.780) recall: (test=0.622) total time=   9.7s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.8s remaining:    0.0s


[CV] END  f1: (test=0.677) precision: (test=0.748) recall: (test=0.619) total time=  12.4s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   22.2s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   36.6s remaining:    0.0s


[CV] END  f1: (test=0.698) precision: (test=0.770) recall: (test=0.638) total time=  11.1s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   47.7s remaining:    0.0s


[CV] END  f1: (test=0.669) precision: (test=0.746) recall: (test=0.606) total time=  16.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min finished


In [12]:
pd.DataFrame(scores)

,baseline
fit_time,12.800853
score_time,0.091049
test_f1,0.683593
test_precision,0.754707
test_recall,0.625029


In [13]:
pipe_base.fit(data,target)

/home/charubaiel/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=6, class_weight={0.0: 10940, 1.0: 3619},
                                    max_iter=1000))])

### work with text

In [8]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
stopwords = nltk.corpus.stopwords.words('russian')

In [32]:
def normalizer (text):
    words_only = re.sub('[^А-я]+',' ',text.lower())
    doc = Doc(words_only)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    clean_text = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if (token.lemma not in stopwords) & (len(set(token.lemma))>1):
            clean_text.append(token.lemma)
            
    return ' '.join(clean_text)

In [33]:
raw_data = data.copy()
old_text = ' жожо '.join(data)
new_text = normalizer(old_text)
data = pd.Series(new_text.split('жожо'))
raw_val_data = val_data.copy()
val_data = pd.Series(normalizer(' жожо '.join(val_data)).split('жожо'))

In [17]:
clf_norm = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts(normalize=True).to_dict())
pipe_norm = pipeline.make_pipeline(tf,clf_norm)
scores['normalize'] = pd.DataFrame(model_selection.cross_validate(pipe_norm,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [18]:
pipe_norm.fit(data,target)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=6,
                                    class_weight={0.0: 0.7514252352496738,
                                                  1.0: 0.24857476475032625},
                                    max_iter=1000))])

In [19]:
pd.DataFrame(scores)

,baseline,normalize
fit_time,12.800853,0.653476
score_time,0.091049,0.059086
test_f1,0.683593,0.441161
test_precision,0.754707,0.929528
test_recall,0.625029,0.289309


### micro EDA

In [20]:
eda = vk_all.copy(deep=True)

In [21]:
eda['txt_len'] = eda['comment'].str.len()
eda['txt_len_avg'] = eda['comment'].str.split().apply(lambda x: np.mean([len(word) for word in x]))
eda['txt_words'] = eda['comment'].str.count(' ')
eda['txt_puncts'] = eda['comment'].str.count('[^\w^ ]')
eda['txt_upper_cnt'] = eda['comment'].apply(lambda x: len([i for i in x if i.isupper()]))
eda['txt_pct_upper'] = eda['txt_upper_cnt'] / eda['txt_len']
eda['txt_pos_punc'] = eda['comment'].apply(lambda text: len(re.findall(r'\)|D',text)))
eda['txt_neg_punc'] = eda['comment'].apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))

In [22]:
eda.groupby('toxic')[eda.filter(regex='txt').columns].agg(['mean','std'])

txt_len             txt_len_avg            txt_words             \
             mean         std        mean       std       mean        std   
toxic                                                                       
0.0    194.213332  274.750067    5.312308  1.063207  29.713436  40.811415   
1.0    141.392665  261.776417    5.467866  1.466984  21.449233  42.106635   

      txt_puncts           txt_upper_cnt            txt_pct_upper            \
            mean       std          mean        std          mean       std   
toxic                                                                         
0.0     7.481849  9.135183      3.984978   7.812980      0.023631  0.027502   
1.0     5.972234  9.188326      6.595939  26.932055      0.052774  0.130923   

      txt_pos_punc           txt_neg_punc            
              mean       std         mean       std  
toxic                                                
0.0       0.279261  0.709825     0.499478  1.353467  
1.0       0.094488  0.434023     0.454414  1.689128

In [23]:
eda.iloc[:,1:].corr()

,toxic,txt_len,txt_len_avg,txt_words,txt_puncts,txt_upper_cnt,txt_pct_upper,txt_pos_punc,txt_neg_punc
toxic,1.000000,-0.091782,0.060394,-0.094138,-0.077608,0.072997,0.171514,-0.136895,-0.014424
txt_len,-0.091782,1.000000,0.024703,0.991756,0.924170,0.370052,-0.047513,0.382291,0.529863
txt_len_avg,0.060394,0.024703,1.000000,-0.025150,0.016616,0.022158,0.079450,-0.001665,0.037176
txt_words,-0.094138,0.991756,-0.025150,1.000000,0.929034,0.368631,-0.049015,0.378389,0.514993
txt_puncts,-0.077608,0.924170,0.016616,0.929034,1.000000,0.346096,-0.033393,0.418016,0.497885
txt_upper_cnt,0.072997,0.370052,0.022158,0.368631,0.346096,1.000000,0.565105,0.115176,0.692265
txt_pct_upper,0.171514,-0.047513,0.079450,-0.049015,-0.033393,0.565105,1.000000,-0.040094,0.312005
txt_pos_punc,-0.136895,0.382291,-0.001665,0.378389,0.418016,0.115176,-0.040094,1.000000,0.395420
txt_neg_punc,-0.014424,0.529863,0.037176,0.514993,0.497885,0.692265,0.312005,0.395420,1.000000


### features

In [24]:
def features_from_text(df):
    df = pd.DataFrame(df)
    df.columns=['text']
    df['txt_len'] = df['text'].str.len()
    df['txt_len_avg'] = df['text'].str.split().apply(lambda x: np.mean([len(word) for word in x]))
    df['txt_words'] = df['text'].str.count(' ')
    df['txt_puncts'] = df['text'].str.count('[^\w^ ]')
    df['txt_upper_cnt'] = df['text'].apply(lambda x: len([i for i in x if i.isupper()]))
    df['txt_pos_punc'] = df['text'].apply(lambda text: len(re.findall(r'\)|D',text)))
    df['txt_neg_punc'] = df['text'].apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))
    
    return df.drop('text',axis=1).fillna(0)

In [25]:
fe_tf = feature_extraction.text.TfidfVectorizer()
fe_tf.fit(data,target)
new_fe=preprocessing.FunctionTransformer(features_from_text)
text_preproc = pipeline.FeatureUnion([('idf',fe_tf),('fe',new_fe)])

In [26]:
clf_fe = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts().to_dict())
pipe_fe = pipeline.Pipeline([('preproc',text_preproc),('clf',clf_fe)])

In [27]:
scores['features'] = pd.DataFrame(model_selection.cross_validate(pipe_fe,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _me

In [28]:
pipe_fe.fit(data,target)

/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preproc',
                 FeatureUnion(transformer_list=[('idf', TfidfVectorizer()),
                                                ('fe',
                                                 FunctionTransformer(func=<function features_from_text at 0x7f0c26d025e0>))])),
                ('clf',
                 LogisticRegression(C=6, class_weight={0.0: 10940, 1.0: 3619},
                                    max_iter=1000))])

In [29]:
pd.DataFrame(scores)

,baseline,normalize,features
fit_time,12.800853,0.653476,9.701039
score_time,0.091049,0.059086,0.137960
test_f1,0.683593,0.441161,0.708981
test_precision,0.754707,0.929528,0.756540
test_recall,0.625029,0.289309,0.667313


### vectors

In [17]:
from navec import Navec

In [18]:
nav = Navec.load('models/emb_navec.tar')

In [ ]:
def get_sentence_vecror(sentence_list):
    vectors = []
    for sentence in sentence_list:
        sent_vec = []
        for i in sentence.split():
            if i in nav:
                sent_vec.append(nav[i])
            else:
                sent_vec.append(nav['<unk>'])
        if sentence.strip() == '':
            sent_vec = [nav['<unk>']]
        vectors.append(np.mean(sent_vec,axis=0))
    return np.vstack(vectors)

In [ ]:
vec_func = preprocessing.FunctionTransformer(get_sentence_vecror)

In [ ]:
clf_vec = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts().to_dict())
pipe_vec = pipeline.make_pipeline(vec_func,clf_vec)

In [ ]:
scores['sample_vec'] = pd.DataFrame(model_selection.cross_validate(pipe_vec,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [ ]:
pipe_vec.fit(data,target)

In [ ]:
pd.DataFrame(scores)

### combo

In [38]:
word_vectorizer = feature_extraction.text.TfidfVectorizer()
word_vectorizer.fit(data)

TfidfVectorizer()

In [39]:
char_vectorizer = feature_extraction.text.TfidfVectorizer(
    min_df=3,
    sublinear_tf=True,
    analyzer='char',
    ngram_range=(2,4))
char_vectorizer.fit(data)

TfidfVectorizer(analyzer='char', min_df=3, ngram_range=(2, 4),
                sublinear_tf=True)

In [40]:
idf_fu = pipeline.FeatureUnion([('idf_w',word_vectorizer),('idf_c',char_vectorizer)])

In [41]:
clf_2idf = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts(normalize=True).to_dict())
pipe_idf_fe = pipeline.Pipeline([('idf',idf_fu),('clf',clf_2idf)])

In [42]:
scores['idf_features'] = pd.DataFrame(model_selection.cross_validate(pipe_idf_fe,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [45]:
pipe_idf_fe.fit(data,target)

Pipeline(steps=[('idf',
                 TfidfVectorizer(analyzer='char', min_df=3, ngram_range=(2, 4),
                                 sublinear_tf=True)),
                ('clf',
                 LogisticRegression(C=6,
                                    class_weight={0.0: 0.7514252352496738,
                                                  1.0: 0.24857476475032625},
                                    max_iter=1000))])

In [46]:
pd.DataFrame(scores)

,idf_features,char_features
fit_time,4.217547,2.552681
score_time,0.447093,0.371240
test_f1,0.620917,0.557309
test_precision,0.937239,0.949638
test_recall,0.464488,0.394579


### blend models

In [ ]:
vec_func = preprocessing.FunctionTransformer(get_sentence_vecror)

In [ ]:
class_prior=target.value_counts(normalize=True).values[::-1]

In [ ]:
clf_lr_vec = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts().to_dict())
clf_nb_vec = naive_bayes.BernoulliNB(class_prior=class_prior)
clf_knn_vec = neighbors.KNeighborsClassifier(30)
clf_svc_vec = linear_model.SGDClassifier(loss='modified_huber',class_weight= target.value_counts().to_dict())
clf_rf_vec = lgbm.LGBMClassifier(n_estimators=1500,learning_rate=0.07,num_leaves=15,class_weight= target.value_counts().to_dict())
clf_mlp_vec = neural_network.MLPClassifier(hidden_layer_sizes=(300,1),max_iter=1000,learning_rate='adaptive')


In [ ]:
models = {}
for n,model in tqdm(enumerate([clf_lr_vec,clf_nb_vec,clf_knn_vec,clf_svc_vec,clf_rf_vec,clf_mlp_vec])):
    tmp_pipe = pipeline.make_pipeline(vec_func,model)
    models[n] = pd.DataFrame(model_selection.cross_validate(tmp_pipe,data,target,scoring=['f1','precision','recall'],cv=5)).mean()
    print(models[n])

In [ ]:
pd.DataFrame(models).rename(columns = {0:'lr',1:'nb',2:'knn',3:'svc',4:'gbm',5:'mlp'}).T.sort_values(by='test_f1',ascending=False)

In [ ]:

vote_vec = ensemble.VotingClassifier(estimators=[('lr',clf_lr_vec),('svc',clf_svc_vec),('gbm',clf_rf_vec),('mlp',clf_mlp_vec)],voting='soft')

In [ ]:
pipe_vote_vec = pipeline.make_pipeline(vec_func,vote_vec)

In [ ]:
scores['vote_models'] = pd.DataFrame(model_selection.cross_validate(pipe_vote_vec,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [ ]:
pipe_vote_vec.fit(data,target)

In [ ]:
pd.DataFrame(scores)

### blend results

In [ ]:
def voting(estimators,x,func=np.mean):
    probs = []
    for i in estimators:
        probs.append(i.predict_proba(x)[:,1])
    return np.apply_over_axes(func,np.array(probs),axes=0)[0]

In [ ]:
eval_data = defaultdict(list)
for model in [pipe_norm,pipe_fe,pipe_idf_fe,pipe_vec,pipe_vote_vec]:
    eval_data['f1'].append(metrics.f1_score(val_target,model.predict(val_data)))
    eval_data['recall'].append(metrics.recall_score(val_target,model.predict(val_data)))
    eval_data['precision'].append(metrics.precision_score(val_target,model.predict(val_data)))
    eval_data['roc_auc'].append(metrics.roc_auc_score(val_target,model.predict(val_data)))

In [ ]:
eval_data['f1'].append(metrics.f1_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data).round()))
eval_data['recall'].append(metrics.recall_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data).round()))
eval_data['precision'].append(metrics.precision_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data).round()))
eval_data['roc_auc'].append(metrics.roc_auc_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data).round()))

In [ ]:
eval_data['f1'].append(metrics.f1_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data,func=np.max).round()))
eval_data['recall'].append(metrics.recall_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data,func=np.max).round()))
eval_data['precision'].append(metrics.precision_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data,func=np.max).round()))
eval_data['roc_auc'].append(metrics.roc_auc_score(val_target,voting([pipe_vec,pipe_idf_fe,pipe_vote_vec],val_data,func=np.max).round()))

In [ ]:
pd.DataFrame(eval_data,index=['norm','fe','idf_df','vec','vote_vec','blend_mean','blend_max']).sort_values(by='f1',ascending=False)

### testing

In [ ]:

def check_text (text):
    return pd.Series({'pure_idf_model' : pipe_norm.predict_proba([text])[:,1][0],
    'some_features_model' : pipe_fe.predict_proba([text])[:,1][0],
    'double_idf_model' :pipe_idf_fe.predict_proba([text])[:,1][0],
    'pure_vectors_model' :pipe_vec.predict_proba([text])[:,1][0],
    'blending_models' : pipe_vote_vec.predict_proba([text])[:,1][0],
    'blending_ttl_mean' : voting([pipe_vec,pipe_idf_fe],[text],func=np.mean)[0],
    'blending_ttl_max' : voting([pipe_vec,pipe_idf_fe],[text],func=np.max)[0]},name='Степень токсичности')

In [ ]:
def time_predict(model,text):
    return model.predict_proba([text])[:,1][0]

In [ ]:
%%timeit
time_predict(pipe_norm,'Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал')

In [ ]:
%%timeit
time_predict(pipe_fe,'Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал')

In [ ]:
%%timeit
time_predict(pipe_idf_fe,'Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал')

In [ ]:
%%timeit
time_predict(pipe_vec,'Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал')

In [ ]:
%%timeit
time_predict(pipe_vote_vec,'Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал')

In [ ]:
%%timeit
check_text('Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал')

In [ ]:
check_text('Ну в целом я хочу сказать что товар не плохой, а просто каличный - я рот его ебал').to_frame().style.format('{:.2%}')

### final

In [ ]:
import joblib

In [ ]:
joblib.dump(pipe_vec,'models/model_vec_2.joblib')
joblib.dump(pipe_idf_fe,'models/model_idf_2.joblib')
joblib.dump(pipe_vote_vec,'models/model_vec_vote_2.joblib')